# Notebook 4
This notebook is to append to or format anything in the master table

In [1]:
%pylab inline
from astropy.table import Table, hstack,vstack, Column, QTable
from astropy.io import fits
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import FK5
from astropy.time import Time
from astropy.coordinates import Angle
from astropy.io import ascii
import numpy.ma as ma

Populating the interactive namespace from numpy and matplotlib


In [2]:
### use the master table here 
tsc = Table.read('/Users/JanviMadhani/thaw_star_catalog/subtables/master.csv') ### change file path to your computer location

In [3]:
tsc.colnames

['Thaw Number',
 'YPC Number',
 'YPC Component',
 'Thaw Name',
 'YPC Name',
 'BD Number',
 'Thaw RA1900',
 'Thaw Dec1900',
 'YPC RA1900',
 'YPC Dec1900',
 'RA1950',
 'Dec1950',
 'RA2000',
 'Dec2000',
 'Thaw B',
 'YPC (B-V)',
 'YPC V',
 'Thaw Spectrum',
 'YPC Spectrum',
 'Thaw Rel. Parallax',
 'Thaw Rel. Parallax Error',
 'Delta Pi',
 'Thaw Abs. Parallax',
 'Yale Abs. Parallax',
 'Yale Abs. Parallax Error',
 'Yale Tot. PM',
 'Yale PM Position Angle',
 'No. of Thaw Observations',
 'Thaw Footnotes',
 'Index Card Match Entry',
 'Index Card Field Name',
 'Index Card Mag.',
 'Index Card RA',
 'Index Card Dec',
 'Index Card Equinox',
 'Observations']

In [4]:
### add in column for date of publication
pub_date = Column(np.zeros(len(tsc)))
tsc.add_column(pub_date, name='Date of Publication')
tsc['Date of Publication'] = tsc['Date of Publication'].astype(object)

### Incorporate Data Entry 1

In [ ]:
# write a function to add together two hour angles to go from 1900 to 2000 coordinates

#first convert to arcminutes
def arcminutes(hourangle):
    if not ma.is_masked(hourangle):
        if len(hourangle.split()) >= 3:

            h,m,s = hourangle.split()[0],hourangle.split()[1],hourangle.split()[2] #or degree, 1hr = 60', 1 deg = 60'
            #get rid of +/- in front of dec degrees
            if h[0].isdigit():
                hour = h
            else:
                hour = h[1:]
                

            hour = float(hour)
            minute = float(m)
            sec = float(s)


            minutes = (hour*60) + (minute) + (sec/60)
            
        elif len(hourangle.split()) >= 2:
            m,s = hourangle.split()[0],hourangle.split()[1]
            
            if m[0].isdigit():
                minute = m
            else:
                minute = m[1:]
                
            minute = float(m)
            sec = float(s)
            
            minutes = (minute) + (sec/60)
        else:
            minutes = 0
    else:
        minutes = 0
    return float("%.2f" % minutes)

        
        
def add_hourangle(c1900,secval):
    """
    parameters: 1900 coordinates (RA or Dec) [hh mm ss]
                secular variation in coordinates over 100 years (RA or Dec) [hh mm ss]
    returns: 2000 coordinates [hh mm ss]
    """
    if not ma.is_masked(c1900):
        c1900 = str(c1900)
        secval = str(secval)
        
        ### get the signs out front
        if not c1900[0].isdigit():
            sign1 = c1900[0]
            if sign1 == '-':
                sign1 = '-1'
            else:
                sign1 = '1'
            c1900 = c1900[1:]
        else:
            sign1 = '1'
        if not secval[0].isdigit():
            sign2 = secval[0]
            if sign2 == '-':
                sign2 = '-1'
            else:
                sign2 = '1'
            secval = secval[1:]
        else:
            sign2 = '1'
        
        min_1900 = arcminutes(c1900)
        min_secval = arcminutes(secval)
        
        
        min_2000 = (float(sign1)*min_1900) + (float(sign2)*min_secval)
        
        print(min_2000)
        
        hours = "%.2f" % (float(min_2000)/60)
        
        minutes = "%.2f" % (float(min_2000) % 60)
        
        seconds = "%.2f" % ((float(min_2000) * 60) % 60)
        
        hours = str(hours)
        minutes = str(minutes)
        seconds = str(seconds)
        
        hours = hours.zfill(5)[:2]
        minutes = minutes.zfill(5)[:2]
    
        
        hourangle = hours + ' ' + minutes + ' '+ seconds
    else:
        hourangle = ''
        
    
    
    return hourangle
    

In [ ]:
fake1900 = '00 00 12.776'
fakesecval = '05 09.209'
print(add_hourangle(fake1900,fakesecval))

## Incorporate Data Entry 2 ( Delta Pi Values)
Assuming a 1-to-1 match between TSC number and Delta Pi Values

In [5]:
delta_pi_vals = Table.read('filepath_here') # <--- INSERT file path to csv file with one column with TSC number 
                                            # and second column with delta pi value



In [ ]:
###fill out master table with delta pi values matching based on tsc number between the two files
for i in range(len(tsc)):
    master_thaw_no = tsc['Thaw Number'][i]
    dp_thaw_no = delta_pi_vals['Thaw Number'][i] #CHANGE the name here to the name of the column that has TSC no
    
    if str(master_thaw_no) == str(dp_thaw_no): #look for the matching case between both files
        tsc['Delta Pi'][i] = delta_pi_vals['Delta Pi'][i] #sets the value in the master table,
                                                          #CHANGE the name of the column that holds the delta pi value here

In [ ]:
#CHECK to make sure values filled out as expected
tsc.show_in_notebook()

In [6]:
### update francis' table
### incorporate new parallax values

